Import library

In [5]:
from google.cloud import storage
from google.cloud import bigquery
import numpy as np
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery
from datetime import datetime

In [6]:

def create_mart(query,table_name,dataset_name,project_id,write_disposition='WRITE_TRUNCATE'):
    try:
        start = datetime.now()
        client = bigquery.Client(project = project_id)
        job_config = bigquery.QueryJobConfig()
        job_config.allow_large_results = True
        dest_dataset_ref = client.dataset(dataset_name)
        print('table name ',table_name)
        dest_table_ref = dest_dataset_ref.table(table_name)
        job_config.destination = dest_table_ref
        job_config.create_disposition = 'CREATE_IF_NEEDED'
        job_config.write_disposition = write_disposition
        print('loaded params in '+str(datetime.now()-start))
        #print(query)
        query_job = client.query(query, job_config=job_config)
        print('query fired in '+str(datetime.now()-start))
        query_job.result()  # Wait for job to complete
        print('completed in '+str(datetime.now()-start))
    except Exception as e:
        print('failed - '+str(e))

def read_gbq_schema(table_name, schema_name, project_id):
              #read schema from big query
              client = bigquery.Client(project=project_id)
              dataset_id = schema_name
              dataset_ref = client.dataset(dataset_id)
              dataset = client.get_dataset(dataset_ref)
              table_ref = dataset.table(table_name)
              table = client.get_table(table_ref)
              field_names = [field.name for field in table.schema]
              field_types = [field.field_type for field in table.schema]
              table_schema = pd.DataFrame({'columns':field_names,'data_types':field_types})
              return table_schema


Define project ids and existing table details

In [7]:
project_id ='sky-uk-ids-analytics-prod'
dataset='AAL87'
table='merged_table_subset_GBR_subset'
df=read_gbq_schema(table,dataset,project_id)

In [8]:
df = df[df['data_types']=='STRING']

In [9]:
string_cols=df['columns'][(df.data_types == 'STRING') ].values.tolist()

In [30]:
query= [''' count(distinct {0}) as {0}
'''.format(col,col.replace(' ','_').lower()) for col in string_cols]

In [31]:
query= ','.join(query)

Make edits to your query to find count distinct of all string columns

In [ ]:
query= ' select ' + query + '''from `sky-uk-ids-analytics-prod.AAL87.merged_table_subset_GBR_subset`
where SPECIAL_TEAM_TYPE in ('STD','DAT') AND DISH_MAKE_MODEL  IS NOT NULL'''

In [33]:
query=query.replace('\n', ' ').replace('\r', '')

In [35]:
%time data = pd.read_gbq(query,project_id,dialect = 'standard')

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 810 ms


In [45]:
data2 = data.T.reset_index()

In [46]:
data2.columns = ['Variable_name','Count_distinct']

In [52]:
df2 = df.merge(data2,left_on='columns',right_on='Variable_name')

In [57]:
df = df2[df2['Count_distinct']<1000]

Second query to input only relevant fields

In [71]:
string_cols=df['columns'][(df.data_types == 'STRING') ].values.tolist()

In [85]:
query2= ['''(select "{0}" as  Variable_name,{0} as Variable_value,
round(avg(case when SPECIAL_TEAM_TYPE = "DAT" then 1 else 0 end),3) as target,
count(1) as freq, round(count(1) / (select count(1) from `sky-uk-ids-analytics-prod.AAL87.merged_table_subset_GBR_subset`) ,3) as freq_pct  
from `sky-uk-ids-analytics-prod.AAL87.merged_table_subset_GBR_subset`
where SPECIAL_TEAM_TYPE in ('STD','DAT') AND DISH_MAKE_MODEL  IS NOT NULL
group by 1,2)
'''.format(col,col.replace(' ','_').lower()) for col in string_cols]

In [87]:
query2= 'union all'.join(query2)

In [88]:
query2=query2.replace('\n', ' ').replace('\r', '')

In [90]:
#create_mart(query,output_table_name,output_dataset_name,project_id,write_disposition='WRITE_TRUNCATE')
create_mart(query2,'sample','RSI33',project_id,write_disposition='WRITE_TRUNCATE')

('table name ', 'sample')
loaded params in 0:00:00.004595
query fired in 0:00:00.614626
completed in 0:00:06.371882


In [91]:
df2

,columns,data_types,Variable_name,Count_distinct
0,PROPERTY_ID,STRING,PROPERTY_ID,10436362
1,CREATED_BY,STRING,CREATED_BY,61138
2,LAST_MODIFIED_BY,STRING,LAST_MODIFIED_BY,13263
3,BUILDING_TYPE_CODE,STRING,BUILDING_TYPE_CODE,8
4,BUILDING_TYPE_DESC,STRING,BUILDING_TYPE_DESC,9
5,BUILDING_MATERIAL_CODE,STRING,BUILDING_MATERIAL_CODE,7
6,BUILDING_MATERIAL_DESC,STRING,BUILDING_MATERIAL_DESC,8
7,TRANSMISSION_ISSUE,STRING,TRANSMISSION_ISSUE,7
8,TRANSMISSION_ISSUE_DESC,STRING,TRANSMISSION_ISSUE_DESC,7
9,WALL_WIDTH,STRING,WALL_WIDTH,5
